<a href="https://colab.research.google.com/github/achett/Hierarchical-Model/blob/main/Bayesian_Hierarchical_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hierarchicalforecast
!pip install statsforecast
!pip install datasetsforecast
!pip install nixtlats>=0.1.0
!pip install darts
!pip install mlforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.2/508.2 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.85 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.8/279.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.5 MB/s eta 0:00:00
  Created wheel for fugue-sql-antlr: filename=fugue_sql_antlr-0.2.0-py3-none-any.whl size=158196 sha256=0bf00e323411c7247d0fc194692707b40b295cf95f9b38fbbbe3088a8ba41589
  Stored in directory: /root/.cache/pip/wheels/5a/b5/4e/216953a1c711da55de29ed7ecf158b4a5bf32ef93d69ad66dd
Successfully built fugue-s

In [2]:
########################
# PACKAGES
########################
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
from datetime import datetime

from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive, AutoETS, AutoCES, AutoTheta
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import LabelEncoder

from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut, MinTrace, OptimalCombination, ERM, PERMBU, Bootstrap, Normality
from hierarchicalforecast.utils import aggregate
from nixtlats import TimeGPT
os.environ['NIXTLA_ID_AS_COL'] = '1'


from darts import TimeSeries, concatenate
from darts.models import RegressionModel, LightGBMModel, ExponentialSmoothing, StatsForecastAutoETS, StatsForecastAutoARIMA, KalmanForecaster
from darts.dataprocessing.transformers import Scaler
from lightgbm import LGBMRegressor
from darts.metrics import mae, rmse, mape, quantile_loss, mse, ope
from darts.utils.likelihood_models import QuantileRegression

pd.options.display.float_format = '{:,.2f}'.format

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [30]:
##############
# PARAMS
##############
fct_periods = 12
fct_st_date = '2023-04-01'
fct_end_date = '2023-12-01'

# Create hierarchical structure and constraints
hierarchy_levels = [['TopLv'],
                    ['TopLv', 'ProductLv'],
                    ['TopLv', 'ProductLv', 'Lv1'],
                    ['TopLv', 'ProductLv', 'Lv1', 'Lv2'],
                    ['TopLv', 'ProductLv', 'Lv1', 'Lv2', 'Lv3'],
                    ['TopLv', 'ProductLv', 'Lv1', 'Lv2', 'Lv3', 'Lv4'],
                    ['TopLv', 'ProductLv', 'Lv1', 'Lv2', 'Lv3', 'Lv4', 'Lv5']]

inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/regional_hierarchy.xlsx'
sheet_name = 'regional_hierarchy v2'
r_hier = pd.read_excel(inputFile, sheet_name=sheet_name)

inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/model_selection.xlsx'
model_selection = pd.read_excel(inputFile)

In [4]:
##############
# FUNCTIONS
##############
def prepare_data(data, r_hier):
    # Merge hierarchy
    data = data.merge(r_hier, how='left', left_on='cost_object', right_on='Lv5')

    # Transform date and y
    data['ds'] = pd.to_datetime(data['ds'])
    data['y'] = data['y'].astype(float)

    # Address NA values
    data['y'] = data['y'].fillna(0)
    data['TopLv'] = data['TopLv'].fillna('')
    data['Lv1'] = data['Lv1'].fillna('')
    data['Lv2'] = data['Lv2'].fillna('')
    data['Lv3'] = data['Lv3'].fillna('')
    data['Lv4'] = data['Lv4'].fillna('')
    data['Lv5'] = data['Lv5'].fillna('')
    data['product'] = data['product'].fillna('')

    # Create hierarchical dataframe
    data.rename(columns={'product': 'ProductLv'}, inplace=True)
    data = data[['TopLv', 'ProductLv', 'Lv1', 'Lv2', 'Lv3', 'Lv4', 'Lv5', 'ds', 'y']]

    # Replace '/' with '_' in the four columns
    data['TopLv'] = data['TopLv'].str.replace('/', '_')
    data['ProductLv'] = data['ProductLv'].str.replace('/', '_')
    data['Lv1'] = data['Lv1'].str.replace('/', '_')
    data['Lv2'] = data['Lv2'].str.replace('/', '_')
    data['Lv3'] = data['Lv3'].str.replace('/', '_')
    data['Lv4'] = data['Lv4'].str.replace('/', '_')
    data['Lv5'] = data['Lv5'].str.replace('/', '_')

    data['unique_id'] = data['TopLv'] + '/' + data['ProductLv'] + '/' + data['Lv1'] + '/' + data['Lv2'] + '/' + data['Lv3'] + '/' + data['Lv4'] + '/' + data['Lv5']

    return data

def prepare_feature(data, r_hier, volume_act2, feature_name):

    # Select and rename columns
    data = data[['cost_object', 'product', 'ds', feature_name]].rename(columns={feature_name: 'y'})

    # Apply any additional preparation (assuming prepare_data is a function you have defined)
    data = prepare_data(data, r_hier)

    # Rename the columns back
    data = data.rename(columns={'y': feature_name})

    # Merge with the volume_act2 dataframe
    merged_df = data.merge(volume_act2[['unique_id', 'ds']], how='right', on=['unique_id', 'ds'])

    return merged_df


In [24]:
##############
# DATA LOAD
##############
inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/budgetFY23.csv'
budget = pd.read_csv(inputFile)
# budget = budget[budget['category']=='EQUIV_UNIT - Equivalent Units']
budget = budget[budget['category']=='UC110000 - Total Revenue']
budget.rename(columns={'country': 'cost_object'}, inplace=True)
budget = prepare_data(budget, r_hier)

inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/revenue_output.csv'
volume_act = pd.read_csv(inputFile)
volume_act.rename(columns={'value': 'y'}, inplace=True)
volume_act = prepare_data(volume_act, r_hier)

inputFile = '/content/drive/MyDrive/Colab Notebooks/SGA Prediction/data/sga_output.csv'
sga = pd.read_csv(inputFile)

sga1 = prepare_feature(sga, r_hier, volume_act, 'AP')
sga2 = prepare_feature(sga, r_hier, volume_act, 'Field_Sales')

In [27]:
########################
# SAMPLE
########################
# Subset
regs2include = ['Global/ENZA - Enzalutamide/D_USCOM - US Commercial/D_USCOM - US Commercial/D_USCOM - US Commercial/D_USCOM - US Commercial/US10 - Astellas Pharma US, Inc.',  'Global/REGADENOSN - Regadenoson/D_USCOM - US Commercial/D_USCOM - US Commercial/D_USCOM - US Commercial/D_USCOM - US Commercial/US10 - Astellas Pharma US, Inc.']
volume_act = volume_act[volume_act['unique_id'].isin(regs2include)]

# level2include = ['CIS']
# volume_act = volume_act[volume_act['level3'].isin(level2include)]

In [29]:
########################
# IDENTIFY UNIVERSE
########################
tested_ts = set(budget['unique_id'].unique()).intersection(volume_act['unique_id'].unique())

# Find unique IDs present in budget_h but not in rev
unique_ids_in_budget_not_in_rev = set(budget['unique_id'].unique()).difference(volume_act['unique_id'].unique())

# Find unique IDs present in rev but not in budget_h
unique_ids_in_rev_not_in_budget = set(volume_act['unique_id'].unique()).difference(budget['unique_id'].unique())

# Filter volume
volume_act = volume_act[volume_act['unique_id'].isin(tested_ts)]

In [33]:
########################
# SEGMENT TIME SERIES
########################
new_products = ['ENFORTUMAB - Enforumab Vedotin', 'ROXADUSTNT - Roxadustant']
loe_products = ['REGADENOSN - Regadenoson']
div_products = ['MICAFUNGIN - Micafungin Sodium']

new_ids = volume_act[volume_act['ProductLv'].isin(new_products)]['unique_id'].unique().tolist()
loe_ids = volume_act[volume_act['ProductLv'].isin(loe_products)]['unique_id'].unique().tolist()
divested_ids = volume_act[volume_act['ProductLv'].isin(div_products)]['unique_id'].unique().tolist()

# IDs with A&P and Field Sales Spend
grouped1 = sga1.groupby('unique_id')[['AP']].sum()
grouped2 = sga2.groupby('unique_id')[['Field_Sales']].sum()
spend_ids = set(grouped1[(grouped1['AP'] > 0)].index.tolist() + grouped2[(grouped2['Field_Sales'] > 0)].index.tolist())
spend_ids = spend_ids.difference(new_ids + loe_ids + divested_ids)

# IDs with no spend
non_spend_ids = volume_act[~volume_act['unique_id'].isin(spend_ids)]['unique_id'].unique()

# Model Selection
arima_regions = model_selection[model_selection['model']=='ARIMA']['Lv3'].unique()
ets_regions = model_selection[model_selection['model']=='ETS']['Lv3'].unique()
# arima_ids = volume_act[(volume_act['level3'].isin(arima_regions)) & (~volume_act['unique_id'].isin(spend_ids))]['unique_id'].unique().tolist()
# ets_ids = volume_act[(volume_act['level3'].isin(ets_regions)) & (~volume_act['unique_id'].isin(spend_ids))]['unique_id'].unique().tolist()

arima_ids = volume_act[(volume_act['Lv3'].isin(arima_regions))]['unique_id'].unique().tolist()
ets_ids = volume_act[(volume_act['Lv3'].isin(ets_regions))]['unique_id'].unique().tolist()

In [ ]:
# ########################
# # INTERMITTENT DEMAND CANDIDATES
# ########################

# # Function to calculate the percentage of zeros after the first non-zero
# def calculate_percentage_zeros(df):
#     # Find the index of the first non-zero entry in 'y'
#     first_non_zero_index = df.loc[df['y'] != 0].index.min()
#     # If there are no non-zero values, return None or 0 based on your preference
#     if pd.isna(first_non_zero_index):
#         return None  # Or return 0 if you want to treat this as 0% zeros following non-zero
#     # Select the subset of 'y' after the first non-zero
#     post_non_zero_series = df.loc[first_non_zero_index:, 'y']
#     # Count the number of zeros in this subset
#     num_zeros = (post_non_zero_series == 0).sum()
#     # Calculate the percentage of zeros
#     percentage_zeros = num_zeros / len(post_non_zero_series) * 100
#     return percentage_zeros

# # Apply the function to each group and reset index to make unique_id a column
# percentage_zeros_df = volume_act.groupby('unique_id').apply(calculate_percentage_zeros).reset_index(name='percentage_zeros')

# inter_demand_ids = percentage_zeros_df[percentage_zeros_df['percentage_zeros']>=50]['unique_id'].tolist()


In [35]:
########################
# RUN ETS & ARIMA
########################
def convert_fct2df(forecasts):
    forecast_dfs = []
    for unique_id, forecast_ts in forecasts.items():
        df = TimeSeries.quantiles_df(forecast_ts, quantiles=[0.005, 0.025, 0.165, 0.250, 0.500, 0.750, 0.835, 0.975, 0.995])
        df['unique_id'] = unique_id
        df = df.reset_index()
        df = df.rename(columns={'y_0.5': 'y'})
        forecast_dfs.append(df)

    # Concatenate all forecast DataFrames into a single DataFrame
    all_forecasts_df = pd.concat(forecast_dfs, axis=0)

    # Reorder and rename columns as needed
    columns = ['unique_id'] + [col for col in all_forecasts_df.columns if col != 'unique_id']
    all_forecasts_df = all_forecasts_df[columns]

    all_forecasts_df.columns.name = None

    return all_forecasts_df

def generate_time_series_dict(data, fct_periods, filter_data):
    # Split train/test sets
    test = data.groupby('unique_id').tail(fct_periods)
    train = data.drop(test.index)

    # Prepare time series dataframes
    time_series_dfs = {uid: group for uid, group in train.groupby('unique_id')}
    time_series_dict = {}

    if filter_data:
        # Filter out time series with insufficient non-zero data points
        filtered_time_series_dfs = {}
        for uid, group in time_series_dfs.items():
            non_zero_index = group['y'].ne(0).idxmax()
            start_index = max(0, non_zero_index - (13 - 1))
            filtered_df = group.loc[non_zero_index:] if group.loc[non_zero_index:].shape[0] >= 13 else group.loc[start_index:]
            if not filtered_df.empty:
                filtered_time_series_dfs[uid] = filtered_df
        # Convert each filtered DataFrame into a Darts TimeSeries object
        time_series_dict = {uid: TimeSeries.from_dataframe(group, 'ds', 'y') for uid, group in filtered_time_series_dfs.items()}
    else:
        # Convert each original DataFrame into a Darts TimeSeries object without filtering
        time_series_dict = {uid: TimeSeries.from_dataframe(group, 'ds', 'y') for uid, group in time_series_dfs.items()}

    return time_series_dict

def generate_forecast(data, fct_periods, model2use, filter_data=True):

    # Use the nested function to generate the time series dictionary
    time_series_dict = generate_time_series_dict(data, fct_periods, filter_data)

    # Create and fit a model for each time series
    models = {}
    for uid, series in time_series_dict.items():
        model = get_model(model2use)
        model.fit(series)
        models[uid] = model

    # Forecasting
    fct_dict = {uid: model.predict(fct_periods, num_samples=20) for uid, model in models.items()}
    # Convert forecasts into a dataframe
    fct_df = convert_fct2df(fct_dict)

    return fct_dict, fct_df

# Function to dynamically get the model instance
def get_model(model_name):
    if model_name == 'AutoETS':
        return StatsForecastAutoETS()
    elif model_name == 'ARIMA':
        return StatsForecastAutoARIMA(season_length=12)
    elif model_name == 'KF':
        return KalmanForecaster(dim_x=12)
    else:
        raise ValueError(f"Unsupported model: {model_name}")

ets_dict, ets_df = generate_forecast(volume_act, fct_periods, model2use='AutoETS', filter_data=True)
arima_dict, arima_df = generate_forecast(volume_act, fct_periods, model2use='ARIMA', filter_data=True)
# kf_fct = generate_forecast(volume_act, fct_periods, model2use='KF', filter_data=True)


In [ ]:
########################
# RUN QUARTERLY MODEL
########################
# Function to resample and sum data by quarter for each group
def resample_group(group):
    group = group.set_index('ds')  # Set 'ds' as the index
    resampled_group = group.resample('Q').agg({'y': 'sum'})  # Aggregate data by quarter
    return resampled_group

quarterly_data = volume_act
quarterly_data['ds'] = pd.to_datetime(quarterly_data['ds'])

# Group by 'unique_id' and apply the resampling function
quarterly_data = quarterly_data.groupby('unique_id').apply(resample_group).reset_index()



In [37]:
########################
# XTREND - DECAY
########################
def apply_exponential_decay(df, start_date, end_date, end_value_percentage, target_unique_ids):
    # Convert 'ds' to datetime if it's not already
    df['ds'] = pd.to_datetime(df['ds'])
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Function to apply decay for each group
    def decay_group(group):
        # Only apply changes if unique_id is in target_unique_ids
        if group['unique_id'].iloc[0] not in target_unique_ids:
            return group

        # Sort by date to ensure proper indexing
        group = group.sort_values(by='ds')

        # Columns to apply decay to
        decay_columns = [col for col in group.columns if col not in ['unique_id', 'ds']]

        # Initialize a dictionary to keep the end values for each decay column
        end_values = {}

        # Find start and end values and dates for each column
        for col in decay_columns:
            if start_date in group['ds'].values and end_date in group['ds'].values:
                start_value = group.loc[group['ds'] == start_date, col].iloc[0]
                end_value = start_value * end_value_percentage
                end_values[col] = end_value  # Store the end value for this column

                # Calculate the decay rate based on exponential decay formula
                days = (end_date - start_date).days
                decay_rate = np.log(end_value / start_value) / days

                # Apply exponential decay for dates between start_date and end_date
                for date in pd.date_range(start_date, end_date):
                    if date in group['ds'].values:
                        t = (date - start_date).days
                        new_value = start_value * np.exp(decay_rate * t)
                        group.loc[group['ds'] == date, col] = new_value

        # Replace column values for dates after end_date with the respective end values
        for col, end_value in end_values.items():
            if end_value is not None:  # Ensure there was an end value calculated
                group.loc[group['ds'] > end_date, col] = end_value

        return group

    # Apply the decay_group function to each group and return the modified dataframe
    return df.groupby('unique_id').apply(decay_group).reset_index(drop=True)

# Apply exponential decay
# lgbm_fct.rename(columns={'LGBM': 'y'}, inplace=True)
ets_df.rename(columns={'ETS': 'y'}, inplace=True)
arima_df.rename(columns={'ARIMA': 'y'}, inplace=True)

# Micafungin
arima_df = apply_exponential_decay(arima_df, '2023-07-01', '2023-08-01', 0, divested_ids)
ets_df = apply_exponential_decay(ets_df, '2023-07-01', '2023-08-01', 0, divested_ids)

# Lexiscan
arima_df = apply_exponential_decay(arima_df, '2023-04-01', '2023-12-01', .1, loe_ids)
ets_df = apply_exponential_decay(ets_df, '2023-04-01', '2023-12-01', .1, loe_ids)


<ipython-input-37-7d582f2577b1>:51: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  return df.groupby('unique_id').apply(decay_group).reset_index(drop=True)
<ipython-input-37-7d582f2577b1>:51: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  return df.groupby('unique_id').apply(decay_group).reset_ind

In [ ]:
########################
# XTREND - GROWTH
########################
# volume_act = volume_act[~volume_act['unique_id'].isin(new_ids)]

In [38]:
########################
# METRICS
########################
# Subset
volume_act_xsm = volume_act[['unique_id', 'ds', 'y']]
budget2 = budget[['unique_id', 'ds', 'y']]
ets_df2 = ets_df[['unique_id', 'ds', 'y']]
arima_df2 = arima_df[['unique_id', 'ds', 'y']]

# Assign names
volume_act_xsm.rename(columns={'y': 'Actuals'}, inplace=True)
budget2.rename(columns={'y': 'Budget'}, inplace=True)
ets_df2.rename(columns={'y': 'ETS'}, inplace=True)
arima_df2.rename(columns={'y': 'ARIMA'}, inplace=True)

# Merge actuals, budget and forecast
rev_at = volume_act_xsm.merge(ets_df2, on=['unique_id', 'ds'], how='left')
rev_at = rev_at.merge(budget2, on=['unique_id', 'ds'], how='left')
rev_at = rev_at.merge(arima_df2, on=['unique_id', 'ds'], how='left')

# Conditions for selection
# conditions = [rev_at['unique_id'].isin(spend_ids),rev_at['unique_id'].isin(arima_ids),rev_at['unique_id'].isin(ets_ids)]
# choices = [rev_at['ARIMA'], rev_at['ARIMA'],rev_at['ETS']]

conditions = [rev_at['unique_id'].isin(arima_ids),rev_at['unique_id'].isin(ets_ids)]
choices = [rev_at['ARIMA'],rev_at['ETS']]

# Creating the new column 'SelectedFCT' based on the conditions
rev_at['SelectedFCT'] = np.select(conditions, choices, default=np.nan)

# Only keep tested ts
rev_at = rev_at[rev_at['unique_id'].isin(tested_ts)]

# Filter for dates
data4metrics = rev_at[(rev_at['ds']<=fct_end_date) & (rev_at['ds']>=fct_st_date)]

# Sum up the values for each unique_id
numeric_cols = data4metrics.columns.drop(['unique_id', 'ds'])
summed_df = data4metrics.groupby('unique_id')[numeric_cols].sum()

# Calculate difference and percentage differences from 'Actuals'
absolute_diff = summed_df.subtract(summed_df['Actuals'], axis=0).abs()
percentage_diff = summed_df.subtract(summed_df['Actuals'], axis=0).div(summed_df['Actuals'], axis=0).abs()

# Drop the 'Actuals' column as we don't need to compare it with itself
absolute_diff.drop(columns=['Actuals', 'ARIMA', 'ETS'], inplace=True)

# Find the column with the lowest difference for each unique_id and add to metrics table
min_diff_col = absolute_diff.idxmin(axis=1)
data4metrics['lowest_diff_col'] = data4metrics['unique_id'].map(min_diff_col)

# Find winner
winner = data4metrics.groupby('lowest_diff_col')

# Get Budget winners
bud_winners = winner.get_group('Budget')['unique_id'].unique()

winner['unique_id'].nunique()

<ipython-input-38-64a0af311e5d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_act_xsm.rename(columns={'y': 'Actuals'}, inplace=True)
<ipython-input-38-64a0af311e5d>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  budget2.rename(columns={'y': 'Budget'}, inplace=True)
<ipython-input-38-64a0af311e5d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ets_df2.rename(columns={'y': 'ETS'}, inplace=True)
<ipython-input-38-64a0af31

lowest_diff_col
Budget         1
SelectedFCT    1
Name: unique_id, dtype: int64

In [39]:
########################
# CREATE PLOT DATA
########################
fct_st_date = pd.to_datetime(fct_st_date)

# Add revenue actuals
data2plot = rev_at.copy()
data2plot['ds'] = pd.to_datetime(data2plot['ds'])

# Update Actuals columns
data2plot['Actuals (Train)'] = data2plot['Actuals'].copy()
data2plot['Actuals'] = data2plot.apply(lambda row: row['Actuals'] if row['ds'] >= fct_st_date else None, axis=1)
data2plot['Actuals (Train)'] = data2plot.apply(lambda row: row['Actuals (Train)'] if row['ds'] < fct_st_date else None, axis=1)

# Filter to end date
data2plot = data2plot[data2plot['ds']<=fct_end_date]

# Find TS to fix
ts2fix = data2plot[data2plot['unique_id'].isin(bud_winners)]
tsnonspend = data2plot[data2plot['unique_id'].isin(non_spend_ids)]

data2plot.head()

,unique_id,ds,Actuals,ETS,Budget,ARIMA,SelectedFCT,Actuals (Train)
0,Global/ENZA - Enzalutamide/D_USCOM - US Commer...,2014-04-01,NaN,NaN,NaN,NaN,NaN,"5,030,183,962.65"
1,Global/ENZA - Enzalutamide/D_USCOM - US Commer...,2014-05-01,NaN,NaN,NaN,NaN,NaN,"4,848,199,687.80"
2,Global/ENZA - Enzalutamide/D_USCOM - US Commer...,2014-06-01,NaN,NaN,NaN,NaN,NaN,"5,213,402,131.35"
3,Global/ENZA - Enzalutamide/D_USCOM - US Commer...,2014-07-01,NaN,NaN,NaN,NaN,NaN,"6,323,371,010.25"
4,Global/ENZA - Enzalutamide/D_USCOM - US Commer...,2014-08-01,NaN,NaN,NaN,NaN,NaN,"6,160,038,163.65"


In [65]:
########################
# CREATE HIERARCHICAL DATAFRAMES
########################
def split_unique_id_into_columns(df, column_name):

    # Split 'unique_id' into 4 new columns
    split_columns = df[column_name].str.split('/', expand=True)

    # Rename the columns
    split_columns.columns = hierarchy_levels[-1]

    # Join back to original dataframe
    result_df = pd.concat([df, split_columns], axis=1)

    # Check for columns that are not in split_columns, 'unique_id' or 'ds'
    for col in result_df.columns:
        if col not in hierarchy_levels[-1] + ['unique_id', 'ds']:
            # Rename the column to 'y'
            result_df = result_df.rename(columns={col: 'y'})
            break  # Assuming only one column needs to be renamed

    return result_df

def create_hier(data, hierarchy_levels, tested_ts, fct_periods):

    # Filter data based on tested_ts
    data_filtered = data[data['unique_id'].isin(tested_ts)]

    # Identify the columns to aggregate
    columns_to_aggregate = [col for col in data_filtered.columns if col not in (hierarchy_levels[-1] + ['unique_id', 'ds'])]

    hier_final = data_filtered[['unique_id', 'ds']]
    # Fill NA values for these columns
    for col in columns_to_aggregate:
        data_quantile = data_filtered[(hierarchy_levels[-1] + ['unique_id', 'ds']+[col])]
        data_quantile.rename(columns={col: 'y'}, inplace=True)
        data_quantile['y'] = data_quantile['y'].fillna(0)

        hier, S_df, tags = aggregate(df=data_quantile, spec=hierarchy_levels)

        hier = hier.reset_index()
        hier.rename(columns={'y': col}, inplace=True)
        hier_final = hier_final.merge(hier, on = ['unique_id', 'ds'], how = 'right')
    # Split the data into train and test sets
    test = hier_final.groupby('unique_id').tail(fct_periods)
    train = hier_final.drop(test.index)

    return train, test, S_df, tags

# Create hierarchies for forecast, actuals and budget
rev_fct = split_unique_id_into_columns(rev_at[rev_at['ds']>=fct_st_date][['unique_id', 'ds', 'SelectedFCT']], 'unique_id')
rev_act = split_unique_id_into_columns(rev_at[['unique_id', 'ds', 'Actuals']], 'unique_id')
rev_bud = split_unique_id_into_columns(rev_at[rev_at['ds']>=fct_st_date][['unique_id', 'ds', 'Budget']], 'unique_id')

revf_train, revf_test, S_df, tags = create_hier(rev_fct, hierarchy_levels, tested_ts, fct_periods)
reva_train, reva_test, S_df, tags = create_hier(rev_act, hierarchy_levels, tested_ts, fct_periods)
bud_train, bud_test, S_df, tags = create_hier(rev_bud, hierarchy_levels, tested_ts, fct_periods)

                                             unique_id         ds  \
0                                               Global 2023-04-01   
1                                               Global 2023-05-01   
2                                               Global 2023-06-01   
3                                               Global 2023-07-01   
4                                               Global 2023-08-01   
..                                                 ...        ...   
112  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-08-01   
113  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-09-01   
114  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-10-01   
115  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-11-01   
116  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-12-01   

                    y  
0   32,352,428,938.30  
1   30,650,204,797.06  
2   29,170,000,839.08  
3   28,056,018,245.96  
4   28,186,759,244.23  
..                ...  
112

In [68]:
########################
# HIERARCHICAL RECONCILIATION FOR POINT FORECAST
########################
# Select reconcilers
reconcilers = [
    TopDown(method='forecast_proportions')
    # OptimalCombination(method = 'ols', nonnegative=True)
    # BottomUp()
    # ERM(method='closed')
]

# Rename y to model name
revf_test.rename(columns={'y': 'model'}, inplace=True)

# Reconcile the base predictions
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
revf_rec = hrec.reconcile(Y_hat_df=revf_test.set_index('unique_id'), Y_df=reva_train.set_index('unique_id'),
                          S=S_df, tags=tags)

# Reset Index and columns
revf_rec = revf_rec[['ds', revf_rec.columns[2]]]
revf_rec = revf_rec.reset_index()
revf_rec.columns = ['unique_id', 'ds', 'Forecast_H']

In [70]:
########################
# HIERARCHICAL RECONCILIATION FOR PROBABILISTIC FORECAST
########################
quantiles = [0.005, 0.025, 0.165, 0.250, 0.500, 0.750, 0.835, 0.975, 0.995] # Define your quantiles
weights = {0.005: 1, 0.025: 1, 0.165: 1, 0.250: 1, 0.500: 1, 0.750: 1, 0.835: 1, 0.975: 1, 0.995: 1}

# Get ranges
Y_hier_df, S_df, tags = aggregate(df=volume_act, spec=hierarchy_levels)
Y_hier_df = Y_hier_df.reset_index()

#split train/test sets
Y_test_df  = Y_hier_df.groupby('unique_id').tail(fct_periods)
Y_train_df = Y_hier_df.drop(Y_test_df.index)

# Compute base predictions
fcst = StatsForecast(df=Y_train_df,
                     models=[AutoETS(season_length=12)],
                     freq='MS', n_jobs=-1)

Y_hat_df = fcst.forecast(h=fct_periods, fitted=True, level=quantiles)

Y_fitted_df = fcst.forecast_fitted_values()
Y_fitted_df = Y_fitted_df[['unique_id', 'ds', 'y', 'AutoETS', 'AutoETS-lo-0.995','AutoETS-lo-0.975', 'AutoETS-lo-0.835', 'AutoETS-lo-0.75', 'AutoETS-hi-0.75', 'AutoETS-hi-0.835','AutoETS-hi-0.975', 'AutoETS-hi-0.995']]
Y_fitted_df.columns = ['unique_id', 'ds', 'y', 'model', 'model-lo-99.5','model-lo-97.5', 'model-lo-83.5', 'model-lo-75', 'model-hi-75', 'model-hi-83.5','model-hi-97.5', 'model-hi-99.5']

# Create probabilistic dataframe
arima_df.rename(columns={'ARIMA': 'y'}, inplace=True)
ets_df.rename(columns={'ETS': 'y'}, inplace=True)

arima_dfp = arima_df[arima_df['unique_id'].isin(arima_ids)]
ets_dfp = ets_df[ets_df['unique_id'].isin(ets_ids)]

rev_prb = pd.concat([arima_dfp, ets_dfp])

# Split 'unique_id' into 4 new columns
split_columns = rev_prb['unique_id'].str.split('/', expand=True)

# Rename the columns
split_columns.columns = hierarchy_levels[-1]
rev_prb = pd.concat([rev_prb, split_columns], axis=1)

# Filter
rev_prb = rev_prb[rev_prb['unique_id'].isin(tested_ts)]

rev_prb_train, rev_prb_test, S_df, tags = create_hier(rev_prb, hierarchy_levels, tested_ts, fct_periods)


reconcilers = [
    BottomUp(),
# TopDown(method='forecast_proportions')
]

# Rename y to model name
rev_prb_test.columns = ['unique_id', 'ds', 'model-lo-99.5','model-lo-97.5', 'model-lo-83.5', 'model-lo-75', 'model', 'model-hi-75', 'model-hi-83.5','model-hi-97.5', 'model-hi-99.5']

Y_fitted_df = Y_fitted_df[Y_fitted_df['unique_id'].isin(rev_prb_test['unique_id'])]

hrec = HierarchicalReconciliation(reconcilers=reconcilers)
rev_prb_rec = hrec.reconcile(Y_hat_df=rev_prb_test.set_index('unique_id'), Y_df=Y_fitted_df.set_index('unique_id'),
                          S=S_df, tags=tags,level=[75, 83.5, 97.5, 99.5], intervals_method='normality')

rev_prb_rec.rename(columns={'model': 'Forecast'}, inplace=True)

rev_prb_rec = rev_prb_rec.reset_index()

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:399: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:399: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
<ipython-input-65-e935f4d265b4>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_quantile.rename(columns={col: 'y

                                             unique_id         ds  \
0                                               Global 2023-01-01   
1                                               Global 2023-02-01   
2                                               Global 2023-03-01   
3                                               Global 2023-04-01   
4                                               Global 2023-05-01   
..                                                 ...        ...   
151  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-08-01   
152  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-09-01   
153  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-10-01   
154  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-11-01   
155  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-12-01   

              y_0.005  
0   25,347,702,104.63  
1   23,694,981,391.73  
2   21,221,525,189.45  
3   25,615,000,275.06  
4   25,823,140,321.04  
..                ...  
151

<ipython-input-65-e935f4d265b4>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_quantile.rename(columns={col: 'y'}, inplace=True)
<ipython-input-65-e935f4d265b4>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_quantile['y'] = data_quantile['y'].fillna(0)


                                             unique_id         ds  \
0                                               Global 2023-01-01   
1                                               Global 2023-02-01   
2                                               Global 2023-03-01   
3                                               Global 2023-04-01   
4                                               Global 2023-05-01   
..                                                 ...        ...   
151  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-08-01   
152  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-09-01   
153  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-10-01   
154  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-11-01   
155  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-12-01   

               y_0.25  
0   28,266,552,469.81  
1   28,014,661,298.21  
2   28,116,916,850.63  
3   30,283,503,196.68  
4   28,918,358,668.35  
..                ...  
151

<ipython-input-65-e935f4d265b4>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_quantile.rename(columns={col: 'y'}, inplace=True)
<ipython-input-65-e935f4d265b4>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_quantile['y'] = data_quantile['y'].fillna(0)
<ipython-input-65-e935f4d265b4>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_quantile.rename(columns={c

                                             unique_id         ds  \
0                                               Global 2023-01-01   
1                                               Global 2023-02-01   
2                                               Global 2023-03-01   
3                                               Global 2023-04-01   
4                                               Global 2023-05-01   
..                                                 ...        ...   
151  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-08-01   
152  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-09-01   
153  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-10-01   
154  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-11-01   
155  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-12-01   

               y_0.75  
0   32,913,523,813.73  
1   32,616,695,480.02  
2   32,537,504,797.70  
3   35,738,033,671.94  
4   32,272,203,251.10  
..                ...  
151

<ipython-input-65-e935f4d265b4>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_quantile.rename(columns={col: 'y'}, inplace=True)
<ipython-input-65-e935f4d265b4>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_quantile['y'] = data_quantile['y'].fillna(0)
<ipython-input-65-e935f4d265b4>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_quantile.rename(columns={c

                                             unique_id         ds  \
0                                               Global 2023-01-01   
1                                               Global 2023-02-01   
2                                               Global 2023-03-01   
3                                               Global 2023-04-01   
4                                               Global 2023-05-01   
..                                                 ...        ...   
151  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-08-01   
152  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-09-01   
153  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-10-01   
154  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-11-01   
155  Global/REGADENOSN - Regadenoson/D_USCOM - US C... 2023-12-01   

              y_0.975  
0   35,137,303,818.99  
1   35,636,680,599.48  
2   35,645,522,496.92  
3   39,760,153,402.92  
4   37,380,403,405.15  
..                ...  
151

<ipython-input-65-e935f4d265b4>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_quantile.rename(columns={col: 'y'}, inplace=True)
<ipython-input-65-e935f4d265b4>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_quantile['y'] = data_quantile['y'].fillna(0)


In [71]:
########################
# CREATE DATAFRAME TO PLOT
########################
bud_test.rename(columns={'y': 'Budget'}, inplace=True)

# Update Actuals columns
reva = pd.concat([reva_train, reva_test])
reva['Actuals'] = reva.apply(lambda row: row['y'] if row['ds'] >= fct_st_date else None, axis=1)
reva['Actuals (Train)'] = reva.apply(lambda row: row['y'] if row['ds'] < fct_st_date else None, axis=1)

# Update forecast
rev_prb_rec = rev_prb_rec[rev_prb_rec['ds']>=fct_st_date]

# Merge
rev_at_hier = reva.merge(rev_prb_rec[['unique_id', 'ds', 'Forecast']], on=['unique_id', 'ds'], how='left')
rev_at_hier = rev_at_hier.merge(bud_test, on=['unique_id', 'ds'], how='left')
rev_at_hier2plot = rev_at_hier.drop(columns=['y'])

rev_prb_rec = rev_prb_rec[['unique_id', 'ds', 'model-lo-99.5', 'model-lo-97.5', 'model-lo-83.5',
       'model-lo-75', 'Forecast', 'model-hi-75', 'model-hi-83.5',
       'model-hi-97.5', 'model-hi-99.5']]

In [72]:
########################
# METRICS
########################

def sum_of_differences(time_series):
    # Find the index of the first non-zero value
    first_non_zero_index = next((index for index, value in enumerate(time_series) if value != 0), None)

    # Check if there is a non-zero value in the series
    if first_non_zero_index is None:
        return 0  # Return 0 if there are no non-zero values

    # Calculate the sum of the differences after the first non-zero value
    sum_diff = sum(abs(time_series[i] - time_series[i - 1]) for i in range(first_non_zero_index + 1, len(time_series)))

    # Calculate the number of time points after the first non-zero value minus one
    num_points = len(time_series) - first_non_zero_index - 1

    # Avoid division by zero
    if num_points <= 0:
        return 0

    # Return the result
    return np.array((sum_diff / num_points).values())[0][0]


def metrics(actual_data, forecasted_data, quantiles, weights, sample_columns, stochastic=False):
    # Prepare a list to store WSPL results for each unique_id
    results = []

    # Ensure 'ds' is in datetime format
    actual_data['ds'] = pd.to_datetime(actual_data['ds'])

    for unique_id in actual_data['unique_id'].unique():
        try:
            wspl, rmse_metric, rmsse_metric, ope_metric = np.nan, np.nan, np.nan, np.nan

            # Filter the actual data
            actual_values = actual_data[(actual_data['unique_id'] == unique_id) & (actual_data['ds'] >= fct_st_date)][['ds', 'y']].tail(fct_periods)

            actual_ts = TimeSeries.from_dataframe(actual_values.set_index('ds'))

            historical_actuals = actual_data[actual_data['unique_id'] == unique_id][['ds', 'y']].drop(actual_values.index)

            historical_ts = TimeSeries.from_dataframe(historical_actuals.set_index('ds'))

            # Filter the forecasted data
            forecasted_values = forecasted_data[forecasted_data['unique_id']==unique_id]

            forecasted_values = forecasted_values.sort_values('ds')

            # Find the unique time points
            unique_times = forecasted_values['ds'].unique()
            num_times = len(unique_times)

            # Define the number of components and samples
            num_components = 1  # 'y'
            num_samples = len(sample_columns)   # Number of forecast columns

            # Initialize the 3D array
            array_3d = np.zeros((num_times, num_components, num_samples))

            # Fill in the array
            for i, time in enumerate(unique_times):
                # Select the corresponding rows from the DataFrame
                row = forecasted_values[forecasted_values['ds'] == time]
                # Extract the sample values and assign them to the array
                samples = row[sample_columns].to_numpy().reshape(num_samples)
                array_3d[i, 0, :] = samples

            # Convert the 'ds' column to datetime
            forecasted_values['ds'] = pd.to_datetime(forecasted_values['ds'])

            # Create a DatetimeIndex from the 'ds' column
            datetime_index = pd.DatetimeIndex(forecasted_values['ds'])

            forecasted_ts = TimeSeries.from_times_and_values(datetime_index, array_3d)

            # Initialize losses dictionary
            losses = {}

            if stochastic:
                # Calculate quantile loss for each quantile if stochastic is True
                for q in quantiles:
                    try:
                        losses[q] = quantile_loss(actual_ts, forecasted_ts, q)
                    except Exception as e:  # Use appropriate exception handling based on your quantile_loss function
                        print(f"Error calculating quantile loss for {q}: {e}")
                        losses[q] = np.nan
            else:
                # Set all losses to NaN if stochastic is False
                for q in quantiles:
                    losses[q] = np.nan

            wspl = sum(weights[q] * losses.get(q, np.nan) for q in quantiles) / sum(weights.values()) if stochastic else np.nan

            # Calculate RMSE and RMSSE
            mse_metric = mse(actual_ts, forecasted_ts)
            scaled = sum_of_differences(historical_ts)

            rmse_metric = np.sqrt(mse_metric)
            rmsse_metric = np.sqrt((mse_metric / scaled)) if scaled != 0 else np.nan

            # Calculate the overall percentage error
            try:
                ope_metric = ope(actual_ts, forecasted_ts)
            except Exception as e:
                ope_metric = np.nan

            # Append the result to the list
            results.append({'unique_id': unique_id, 'WSPL': wspl, 'RMSE': rmse_metric, 'RMSSE': rmsse_metric, 'OPE': ope_metric})
        except Exception as e:
            # If an error occurs, log it, and append NaN metrics for this unique_id
            print(f"An error occurred for {unique_id}: {e}")
            results.append({'unique_id': unique_id, 'WSPL': np.nan, 'RMSE': np.nan, 'RMSSE': np.nan, 'OPE': np.nan})

    # Convert the list of results into a DataFrame
    results_df = pd.DataFrame(results)
    return results_df


# Usage:
quantiles = [0.005, 0.025, 0.165, 0.250, 0.500, 0.750, 0.835, 0.975, 0.995] # Define your quantiles
weights = {0.005: 1, 0.025: 1, 0.165: 1, 0.250: 1, 0.500: 1, 0.750: 1, 0.835: 1, 0.975: 1, 0.995: 1}
sample_columns_fct = ['model-lo-99.5', 'model-lo-97.5', 'model-lo-83.5', 'model-lo-75','Forecast', 'model-hi-75', 'model-hi-83.5', 'model-hi-97.5','model-hi-99.5']


rev_at_hier = rev_at_hier[rev_at_hier['unique_id'].isin(rev_prb_rec['unique_id'])]

fct_results = metrics(rev_at_hier[['unique_id', 'ds', 'y']], rev_prb_rec, quantiles, weights, sample_columns_fct, stochastic=True)
bud_results = metrics(rev_at_hier[['unique_id', 'ds', 'y']], bud_test, quantiles, weights, ['Budget'], stochastic=False)

<ipython-input-72-d5be9901b93d>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_data['ds'] = pd.to_datetime(actual_data['ds'])
<ipython-input-72-d5be9901b93d>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_data['ds'] = pd.to_datetime(actual_data['ds'])


In [100]:
########################
# RESULTS DATAFRAME
########################
# Ensure 'ds' is in datetime format
rev_at_hier['ds'] = pd.to_datetime(rev_at_hier['ds'])

# Filter the DataFrame for dates within the specified range
results = rev_at_hier[(rev_at_hier['ds'] >= fct_st_date) & (rev_at_hier['ds'] <= fct_end_date)]
results = results.groupby('unique_id')[['Actuals', 'Budget', 'Forecast']].sum().reset_index()

def create_output(results, fct_results, metrics, col_nme):
    # Merge results with forecast results
    results_fct = results[['unique_id', 'Actuals', col_nme]].merge(fct_results, how='left', on='unique_id')

    # Split 'unique_id' into separate levels
    split_columns = results_fct['unique_id'].str.split('/', expand=True)
    split_columns.columns = hierarchy_levels[-1]

    # Concatenate split columns back to the original dataframe
    results_fct = pd.concat([results_fct, split_columns], axis=1)

    # Rearrange and rename columns
    results_fct = results_fct[['unique_id'] + hierarchy_levels[-1] + ['Actuals', col_nme, 'WSPL', 'RMSE', 'RMSSE', 'OPE']]
    results_fct.columns = ['unique_id'] + hierarchy_levels[-1] +  ['Actuals', col_nme, 'SPL', 'RMSE', 'RMSSE', 'Error %']

    for column in hierarchy_levels[-1]:
      results_fct[column] = results_fct[column].fillna('')

    # Filter for Product Level where 'Region' is empty
    results_fct_prod = results_fct[results_fct['Lv1'] == ''][hierarchy_levels[1] + ['Actuals', col_nme, 'RMSE', 'RMSSE', 'Error %', 'SPL']]

    # Initialize a dictionary to hold the pivoted DataFrames for each metric
    pivot_dfs = {}

    for metric in metrics:
        pivot_df = results_fct.pivot_table(
            index=hierarchy_levels[1],
            columns=hierarchy_levels[-1][-5:],
            values=metric,
            aggfunc='sum'  # Change as needed
        ).reset_index()

        # Add the pivoted DataFrame to the dictionary
        pivot_dfs[metric] = pivot_df

    return pivot_dfs, results_fct_prod

results_fct_prod = create_output(results, fct_results, metrics, 'Forecast')
results_fct_prod

# Forecast
metrics = ['Actuals', 'Forecast', 'SPL', 'RMSE', 'RMSSE', 'Error %']
pivot_dfs, results_fct_prod = create_output(results, fct_results, metrics, 'Forecast')
actuals_df = pivot_dfs['Actuals']
output_df = pivot_dfs['Forecast']
rmse_df = pivot_dfs['RMSE']
rmsse_df = pivot_dfs['RMSSE']
ope_df = pivot_dfs['Error %']
spl_df = pivot_dfs['SPL']

# Budget
metrics = ['Actuals', 'Budget', 'SPL', 'RMSE', 'RMSSE', 'Error %']
pivot_dfs2, results_bud_prod = create_output(results, bud_results, metrics, 'Budget')
actuals_df2 = pivot_dfs2['Actuals']
output_df2 = pivot_dfs2['Budget']
rmse_df2 = pivot_dfs2['RMSE']
rmsse_df2 = pivot_dfs2['RMSSE']
ope_df2 = pivot_dfs2['Error %']
spl_df2 = pivot_dfs2['SPL']

# Get column starts
col_starts = [1, results_fct_prod.shape[1]+1, output_df.shape[1]+2, rmse_df.shape[1]+2, rmsse_df.shape[1]+2, ope_df.shape[1]+2]
col_starts_sum = []
running_total = 0

for value in col_starts:
    running_total += value
    col_starts_sum.append(running_total)


# Create a Pandas Excel writer using openpyxl as the engine
filename='/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/consolidated_results.xlsx'
with pd.ExcelWriter(filename) as writer:
    results_fct_prod.to_excel(writer, sheet_name='Sheet1', startrow=3, startcol=col_starts_sum[0], header=True, index=False)
    output_df.to_excel(writer, sheet_name='Sheet1', startrow=2, startcol=col_starts_sum[1], header=True)
    rmse_df.to_excel(writer, sheet_name='Sheet1', startrow=2, startcol=col_starts_sum[2], header=True)
    rmsse_df.to_excel(writer, sheet_name='Sheet1', startrow=2, startcol=col_starts_sum[3], header=True)
    ope_df.to_excel(writer, sheet_name='Sheet1', startrow=2, startcol=col_starts_sum[4], header=True)
    spl_df.to_excel(writer, sheet_name='Sheet1', startrow=2, startcol=col_starts_sum[5], header=True)

    results_bud_prod.to_excel(writer, sheet_name='Sheet2', startrow=3, startcol=col_starts_sum[0], header=True, index=False)
    output_df2.to_excel(writer, sheet_name='Sheet2', startrow=2, startcol=col_starts_sum[1], header=True)
    rmse_df2.to_excel(writer, sheet_name='Sheet2', startrow=2, startcol=col_starts_sum[2], header=True)
    rmsse_df2.to_excel(writer, sheet_name='Sheet2', startrow=2, startcol=col_starts_sum[3], header=True)
    ope_df2.to_excel(writer, sheet_name='Sheet2', startrow=2, startcol=col_starts_sum[4], header=True)
    spl_df2.to_excel(writer, sheet_name='Sheet2', startrow=2, startcol=col_starts_sum[5], header=True)

In [101]:
########################
# PLOT
########################
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import pandas as pd

import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import base64
from io import BytesIO

# Update the function to include filtering based on 'unique_id'
def plot_data(unique_id):
    # Define x_column and y_columns directly
    x_column = data2use.columns[1]
    y_columns = [data2use.columns[2], data2use.columns[3], data2use.columns[4], data2use.columns[5]]

    # Filter data based on selected unique_id
    filtered_data = data2use[data2use['unique_id'] == unique_id]

    # Set up a 1x3 grid of subplots
    fig, (ax1, ax4) = plt.subplots(1, 2, figsize=(25, 5), gridspec_kw={'width_ratios': [4, 1]}) # Adjust layout for table

    # Plotting multiple y-axes on the first subplot
    for y_column in y_columns:
        ax1.plot(filtered_data[x_column], filtered_data[y_column], label=y_column)
    ax1.set_xlabel(x_column)
    ax1.set_ylabel('Values')
    ax1.set_title(f'Revenue for {unique_id}')
    ax1.legend()

    # Remove axis for table
    ax4.axis('off')
    ax4.axis('tight')

    # Displaying the sum table
    display_data = filtered_data[[x_column] + list(y_columns)].copy()
    display_data = display_data[display_data['ds']>=fct_st_date]
    display_data['ds'] = display_data['ds'].dt.strftime('%m/%d/%Y')

    # Create a sum row
    sum_values = {x_column: 'Sum'}
    for col in list(y_columns):
        sum_values[col] = display_data[col].sum()
    sum_row = pd.DataFrame([sum_values])

    # Create a % diff row
    actuals_sum = sum_values['Actuals']
    pdiff_values = {x_column: '% Diff'}
    for col in list(y_columns):
        pdiff_values[col] = ((display_data[col].sum()-actuals_sum) / actuals_sum) * 100 if actuals_sum != 0 else None
        pdiff_values[col] = round(pdiff_values[col], 2)
    perc_diff_row = pd.DataFrame([pdiff_values])

    # Stack the sum row
    display_data = pd.concat([sum_row, display_data], ignore_index=True)

    # Round the values and add commas
    for column in y_columns:
        if column in display_data.columns:
            # Round to two decimal places
            display_data[column] = display_data[column].round(2)
            # Format with commas
            display_data[column] = display_data[column].apply(lambda x: f"{x:,.2f}")

    # Stack the % diff and remove 'Actuals Train'
    display_data = pd.concat([perc_diff_row, display_data], ignore_index=True)
    display_data = display_data.drop('Actuals (Train)', axis=1)

    # Convert perc_diff_data to array for table
    table_data = display_data.to_numpy()
    # Add table at the right
    table = ax4.table(cellText=table_data, colLabels=display_data.columns, loc='right')
    table.auto_set_font_size(False)
    table.set_fontsize(8.5)  # Set smaller font size if necessary
    table.scale(4, 1.8)  # Adjust scale to fit

    plt.tight_layout()
    plt.show()


# data2use = ts2fix
# data2use = tsnonspend
# data2use = data2plot
data2use = rev_at_hier2plot

# Create widgets
unique_id_selector = widgets.SelectionSlider(
    options=data2use['unique_id'].unique(),
    description='unique_id:',
    orientation='horizontal',
    readout=True
)

# Display interactive plot
interact(plot_data, unique_id=unique_id_selector)

interactive(children=(SelectionSlider(description='unique_id:', options=('Global', 'Global/ENZA - Enzalutamide…

<function __main__.plot_data(unique_id)>